In [2]:
# Imports
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

# Install the webdriver_manager pip package in the current Jupyter kernel
# import sys
# !{sys.executable} -m pip install webdriver_manager

In [3]:
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
There is no [mac64] chromedriver for browser  in cache
Trying to download new driver from https://chromedriver.storage.googleapis.com/98.0.4758.102/chromedriver_mac64.zip
Driver has been saved in cache [/Users/matt/.wdm/drivers/chromedriver/mac64/98.0.4758.102]


In [28]:
def start_kayak(city_from, city_to, date_start, date_end):
    """City codes - it's the IATA codes!
    Date format -  YYYY-MM-DD"""
    
    # Starting kayak chrome 
    kayak = ('https://www.kayak.co.uk/flights/' + city_from + '-' + city_to +
             '/' + date_start + '-flexible/' + date_end + '-flexible?sort=bestflight_a')
    driver.get(kayak)

In [68]:
# functions to obtain each property of a dictionary which will then be the exported csv file

"""
{
origin: char(3)
destination: char(3)
time_start: int
time_end: int
date_start: date
date_end: date
price: int
href: string;
}
"""

def scrape_hrefs():
    """
    scrape the hrefs on the page
    """
    href_xpath = '//span[contains(text(), "View Deal")]//parent::a' 
    anchors = driver.find_elements(By.XPATH, href_xpath)
    
    def get_href(anchor):
        return anchor.get_attribute("href")
    
    return list(map(get_href, anchors))

def scrape_prices():
    """
    scrape the prices on the page
    UNFINISHED - perhaps try using conditional statements in XPath
    """
#     price_xpath = '//span[@class="price-text" and contains(@id, "-price-text")]'
    price_xpath = '//a[@class="booking-link "]//descendant::span[@class="price-text"]'
    prices = driver.find_elements(By.XPATH, price_xpath)
    
    return len(prices)

In [29]:
start_kayak("LGW", "CDG", "2022-03-15", "2022-03-22")

In [69]:
# print(scrape_hrefs())
print(scrape_prices())

36
